In [ ]:
import os
import numpy as np                                                                               # Importing numpy for Matrix Operations
import pandas as pd                                                                              # Importing pandas to read CSV files
import matplotlib.pyplot as plt                                                                  # Importting matplotlib for Plotting and visualizing images
import math                                                                                      # Importing math module to perform mathematical operations
import cv2                                                                                       # Importing openCV for image processing
import seaborn as sns 


# Tensorflow modules
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator                              # Importing the ImageDataGenerator for data augmentation
from tensorflow.keras.models import Sequential                                                   # Importing the sequential module to define a sequential model
from tensorflow.keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D,BatchNormalization # Defining all the layers to build our CNN Model
from tensorflow.keras.optimizers import Adam,SGD                                                 # Importing the optimizers which can be used in our model
from sklearn import preprocessing                                                                # Importing the preprocessing module to preprocess the data
from sklearn.model_selection import train_test_split                                             # Importing train_test_split function to split the data into train and test
from sklearn.metrics import accuracy_score, confusion_matrix                                                     # Importing confusion_matrix to plot the confusion matrix

# Display images using OpenCV
#from google.colab.patches import cv2_imshow                                                    # Importing cv2_imshow from google.patches to display images


# Ignore warnings
import warnings
warnings.filterwarnings('ignore')


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
DATADIR = "/kaggle/input/sad-and-happy-face-detection/data"  # Path of data
CATEGORIES = ["happy","sad"]                                # Storing all the categories in categories variable
IMG_SIZE=150  

In [ ]:
# Creating 4 different lists to store the image names for each category by reading them from their respective directories. 
happy_imgs = [fn for fn in os.listdir(f'{DATADIR}/{CATEGORIES[0]}') ]          # Looping over the path of each image from the happy directory
sad_imgs = [fn for fn in os.listdir(f'{DATADIR}/{CATEGORIES[1]}')]            # Looping over the path of each image from the sad directory

     


# Ranodmly selecting 3 images from each category
select_happy = np.random.choice(happy_imgs, 3, replace = False)               
select_sad = np.random.choice(sad_imgs, 3, replace = False)


In [ ]:
# plotting 4 x 3 image matrix
fig = plt.figure(figsize = (10,10))

# Plotting three images from each of the four categories by looping through their path 
for i in range(6):
    if i < 3:
        fp = f'{DATADIR}/{CATEGORIES[0]}/{select_happy[i]}'                    # Here datadir is a path to the training data and categories[0] indicate the first label bread and here we are looping over to take the three random images that we have stored in select_galo variable 
        label = 'Happy'                                                 
    if i>=3 and i<6:
        fp = f'{DATADIR}/{CATEGORIES[1]}/{select_sad[i-3]}'                   # Here datadir is a path to the training data and categories[1] indicate the second label soup and here we are looping over to take the three random images that we have stored in select_menin variable 
        label = 'Sad' 
    ax = fig.add_subplot(4, 3, i+1)
    
    # Plotting each image using load_img function
    fn = tf.keras.preprocessing.image.load_img(fp,target_size = (150,150))
    #fn = image.load_img(fp, target_size = (150,150))
    plt.imshow(fn, cmap='Greys_r')
    plt.title(label)
    plt.axis('off')
plt.show()

## Data Preprocessing

In [ ]:
# Here we will be using a user defined function create_training_data() to extract the images from the directory
data = []                                                             # Storing all the training images
def create_data():
    for category in CATEGORIES:                                                # Looping over each category from the CATEGORIES list
        path = os.path.join(DATADIR,category)                                  # Joining images with labels
        class_num = category                                                   
        for img in os.listdir(path):                                           
          img_array = cv2.imread(os.path.join(path,img))                       # Reading the data
          new_array = cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))                # Resizing the images 
          data.append([new_array,class_num])                          # Appending both the images and labels
create_data() 

In [ ]:
# Creating two different lists to store the Numpy arrays and the corresponding labels
images = []                                                                   
labels = []
np.random.shuffle(data)                                               # Shuffling data to reduce variance and making sure that model remains general and overfit less
for features,label in data:                                           # Iterating over the training data which is generated from the create_training_data() function 
    images.append(features)                                                   # Appending images into X_train
    labels.append(label) 

In [ ]:
# Converting the list into DataFrame
labels = pd.DataFrame(labels, columns=["Label"],dtype=object) 


## EDA

**Checking for Data Imbalance**

In [ ]:
# Storing the value counts of target variable
count=labels.Label.value_counts()
print(count)
print('*'*10)
count=labels.Label.value_counts(normalize=True)
print(count)

In [ ]:
sns.histplot(labels['Label'])
plt.xticks(rotation='vertical')

In [ ]:
# Converting the pixel values into Numpy array
images= np.array(images) 


In [ ]:
images.shape

## Visualizing images using Gaussian Blur 

In [ ]:
# Applying Gaussian Blur to denoise the images
images_gb=[]
for i in range(len(images)):
  # gb[i] = cv2.cvtColor(images[i], cv2.COLOR_BGR2RGB)
  images_gb.append(cv2.GaussianBlur(images[i], ksize =(3,3),sigmaX =  0))

In [ ]:
plt.imshow(images_gb[140])

- It appears that GaussianBlur can be effective because the blurred or denoised image does not seem to remove any relevant information.

## **Splitting the dataset**

- We will only use 10% of our data for testing, 10% of our data for validation and 80% of our data for training.
- We are using the train_test_split() function from scikit-learn. Here, we split the dataset into three parts, train,test and validation.

In [ ]:
from sklearn.model_selection import train_test_split
X_temp, X_test, y_temp, y_test = train_test_split(np.array(images_gb),labels , test_size=0.1, random_state=42,stratify=labels)
X_train, X_val, y_train, y_val = train_test_split(X_temp,y_temp , test_size=0.1, random_state=42,stratify=y_temp)

In [ ]:
y_train.head()

In [ ]:
print(X_train.shape,y_train.shape)
print(X_val.shape,y_val.shape)
print(X_test.shape,y_test.shape)

## Data Normalization

In [ ]:
# Normalizing the image pixels
X_train_normalized = X_train.astype('float32')/255.0
X_val_normalized = X_val.astype('float32')/255.0
X_test_normalized = X_test.astype('float32')/255.0

In [ ]:
print(X_train_normalized.shape,y_train.shape)
print(X_val.shape,y_val.shape)
print(X_test.shape,y_test.shape)

### **Encoding Target Variable**

In [ ]:
from sklearn.preprocessing import LabelBinarizer
enc = LabelBinarizer()
y_train_encoded = enc.fit_transform(y_train)
y_val_encoded=enc.transform(y_val)
y_test_encoded=enc.transform(y_test)

In [ ]:
print(X_train.shape,y_train_encoded.shape)


In [ ]:
model = Sequential()

In [ ]:
model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(150,150,3)))

# Adding max pooling to reduce the size of output of first conv layer
model.add(MaxPooling2D((2, 2), padding = 'same'))

model.add(Conv2D(32, (3, 3), activation='relu', padding="same"))
model.add(MaxPooling2D((2, 2), padding = 'same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding="same"))
model.add(MaxPooling2D((2, 2), padding = 'same'))

model.add(Flatten())
# Adding a fully connected dense layer with 100 neurons    
model.add(Dense(256, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile('adam',loss = tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history_1 = model.fit(
            X_train_normalized, y_train_encoded,
            epochs=30,
            validation_data=(X_val_normalized,y_val_encoded),
            batch_size=16,
            verbose=2
)

In [ ]:
plt.plot(history_1.history['accuracy'])
plt.plot(history_1.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
accuracy = model.evaluate(X_val_normalized, y_val_encoded, verbose=2)

In [ ]:
# Val Prediction 
y_val_pred_ln = model.predict(X_val)
y_val_pred_classes_ln = np.argmax(y_val_pred_ln, axis=1)
normal_y_val = np.argmax(y_val_encoded, axis=1)

In [ ]:
# val Accuracy 
accuracy_score((normal_y_val), y_val_pred_classes_ln)

In [ ]:
accuracy = model.evaluate(X_test_normalized, y_test_encoded, verbose=2)

In [ ]:
# Test Prediction 
y_test_pred_ln = model.predict(X_test)
y_test_pred_classes_ln = np.argmax(y_test_pred_ln, axis=1)
normal_y_test = np.argmax(y_test_encoded, axis=1)

In [ ]:
# Test Accuracy 
accuracy_score((normal_y_test), y_test_pred_classes_ln)

In [ ]:
# Test Accuracy 
accuracy_score((normal_y_test), y_test_pred_classes_ln)

In [ ]:
cf_matrix = confusion_matrix(normal_y_test, y_test_pred_classes_ln)

# Confusion matrix normalized per category true value
cf_matrix_n1 = cf_matrix/np.sum(cf_matrix, axis=1)
plt.figure(figsize=(8,6))
sns.heatmap(cf_matrix_n1, xticklabels=CATEGORIES, yticklabels=CATEGORIES, annot=True)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report((normal_y_test), y_test_pred_classes_ln))

In [ ]:
# Visualizing the predicted and correct label of images from test data
plt.figure(figsize=(2,2))
plt.imshow(X_test[0])
plt.show()
print('Predicted Label', enc.inverse_transform(model.predict((X_test_normalized[0].reshape(1,150,150,3)))))   # reshaping the input image as we are only trying to predict using a single image
print('True Label', enc.inverse_transform(y_test_encoded)[0])                                               # using inverse_transform() to get the output label from the output vector

# Visualizing the predicted and correct label of images from test data
plt.figure(figsize=(2,2))
plt.imshow(X_test[22])
plt.show()
print('Predicted Label', enc.inverse_transform(model.predict((X_test_normalized[22].reshape(1,150,150,3)))))   # reshaping the input image as we are only trying to predict using a single image
print('True Label', enc.inverse_transform(y_test_encoded)[22])                                               # using inverse_transform() to get the output label from the output vector

# Visualizing the predicted and correct label of images from test data
plt.figure(figsize=(2,2))
plt.imshow(X_test[11])
plt.show()
print('Predicted Label', enc.inverse_transform(model.predict((X_test_normalized[11].reshape(1,150,150,3)))))   # reshaping the input image as we are only trying to predict using a single image
print('True Label', enc.inverse_transform(y_test_encoded)[11])                                               # using inverse_transform() to get the output label from the output vector
